# Processing tables to find relevant parts

In [1]:
import torch
import pandas as pd
import numpy as np
import pickle, logging, spacy, sys, os, json, requests
import matplotlib.pyplot as plt

from helpers.classes import Collection
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
import spacy
nlp = spacy.load('en_core_web_trf')

from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
vectorizer = TfidfVectorizer(ngram_range=(2,3))

import nltk
stopwords = set(nltk.corpus.stopwords.words('english'))

In [3]:
from helpers.cloze_generation import generate_clozes_from_point, named_entity_answer_generator as ne_answer_generator, noun_phrase_answer_generator as np_answer_generator

from helpers.table_processing import preprocess_table, read_process_table, find_relevant_column_header, find_relevant_content
from helpers.t5_language_model import summarise_t5_results


df = pd.read_pickle('pickles/dataset_20210625_184837.pkl')
clozes_df = pd.read_json('pickles/clozes_20210807_165700.json')

In [4]:
df

,bulletin,type,point,data
0,businessindustryandtrade/business/businessserv...,date_and_percent,"In 2019, approximate gross value added at basi...",[/businessindustryandtrade/business/businessse...
1,businessindustryandtrade/business/businessserv...,date_and_percent,"The non-financial services sector, which accou...",[/businessindustryandtrade/business/businessse...
2,businessindustryandtrade/business/businessserv...,date_and_percent,Total turnover and purchases of the UK non-fin...,[/businessindustryandtrade/business/businessse...
3,businessindustryandtrade/business/businessserv...,date_and_percent,"Out of the 12 UK regions, 8 regions experience...",[/businessindustryandtrade/business/businessse...
4,businessindustryandtrade/business/businessserv...,date_and_percent,"West Midlands, Yorkshire and The Humber, Scotl...",[/businessindustryandtrade/business/businessse...
...,...,...,...,...
2010,peoplepopulationandcommunity/wellbeing/article...,date_and_percent,Trust in others in their neighbourhood was hig...,[/peoplepopulationandcommunity/wellbeing/datas...
2011,peoplepopulationandcommunity/wellbeing/article...,date_and_percent,Trust in others in their neighbourhood was hig...,[/peoplepopulationandcommunity/wellbeing/datas...
2012,peoplepopulationandcommunity/wellbeing/article...,date_and_percent,People in higher managerial occupations were m...,[/peoplepopulationandcommunity/wellbeing/datas...
2013,peoplepopulationandcommunity/wellbeing/article...,date_and_percent,Around 6 in 10 people (61%) reported feeling s...,[/peoplepopulationandcommunity/wellbeing/datas...


In [5]:
clozes_df

,ids,cloze_text,source_text,answer_text,answer_type,data
0,ccf75a2c157eacc5253f0e8a55b1ded9f5386d58,"In<mask>, approximate gross value added at bas...","In 2019, approximate gross value added at basi...",2019,DATE,[/businessindustryandtrade/business/businessse...
1,fc3346f2545ca5059bdb5150b998300c5001bea5,"In 2019, approximate gross value added at basi...","In 2019, approximate gross value added at basi...","£1,313.9 billion",MONEY,[/businessindustryandtrade/business/businessse...
2,81aa32270d547a368e7b806646c72069ba4fac43,"In 2019, approximate gross value added at basi...","In 2019, approximate gross value added at basi...",£42.8 billion,MONEY,[/businessindustryandtrade/business/businessse...
3,f308eb661ad1d6c8021cf9e22d8e8c7d9c5ff50e,"In 2019, approximate gross value added at basi...","In 2019, approximate gross value added at basi...",3.4%,PERCENT,[/businessindustryandtrade/business/businessse...
4,0a0bcf4b990f1926bc87fa662badadaf7e23d5fa,"In 2019, approximate gross value added at basi...","In 2019, approximate gross value added at basi...",2018,DATE,[/businessindustryandtrade/business/businessse...
...,...,...,...,...,...,...
4085,58116a07356fb4b001d48629d7864909c03fab79,Children who reported “low” satisfaction with ...,Children who reported “low” satisfaction with ...,28.3%,PERCENT,[/peoplepopulationandcommunity/wellbeing/datas...
4086,9902578d5ee75716bec570af2ae7ca8c378f039a,Children who reported “low” satisfaction with ...,Children who reported “low” satisfaction with ...,about 10%,PERCENT,[/peoplepopulationandcommunity/wellbeing/datas...
4087,2124dac9b26f153e90758d9f9f95d51c7aaeebef,9.8% of young people said that they were “ofte...,9.8% of young people said that they were “ofte...,9.8%,PERCENT,[/peoplepopulationandcommunity/wellbeing/datas...
4088,059317f10fe57b868e10900dd413e48a852ed0b3,Those reporting no long-term illness or disabi...,Those reporting no long-term illness or disabi...,44.8%,PERCENT,[/peoplepopulationandcommunity/wellbeing/datas...


# testing kit

In [6]:
_cloze = clozes_df.iloc[20].source_text
_table = clozes_df[clozes_df['source_text'] == _cloze]
_relevant_dfs = _table.data.values[0]

df_iterator = read_process_table(_relevant_dfs[1])
# df_iterator = read_process_table(_relevant_dfs[7])

In [7]:
_relevant_dfs[1].replace('/', '_')[1: ] + '.xls'

'businessindustryandtrade_business_businessservices_datasets_uknonfinancialbusinesseconomyannualbusinesssurveyrevisionsandchangeonpreviousyear.xls'

In [8]:
pd.ExcelFile('datasets/' + _relevant_dfs[1].replace('/', '_')[1:] + '.xls').sheet_names

['Contents', 'ABS Revisions to Data', 'ABS Change on Previous Year']

In [9]:
table = next(df_iterator)
table = next(df_iterator)
# table = next(df_iterator)

IMPUTE_COL_NANS: True
DROP_UNNAMED: True
BACKFILL_HEADERS: True
FILL_NA: True
IMPUTE_FIRST_COL_EMPTY: True
IMPUTE_ALL_COL_EMPTY: True
IMPUTE_COL_NANS: True
DROP_UNNAMED: True
BACKFILL_HEADERS: True
FILL_NA: True
IMPUTE_FIRST_COL_EMPTY: True
IMPUTE_ALL_COL_EMPTY: True


In [10]:
table

,A-S (Part) 1,"Agriculture, Fishing, Production,",Country and Region North East,Total turnover Release 15/05/2020,Total turnover Release 24/06/2021,Total turnover Revision,Total turnover % Revision,Approximate gross value added at basic prices (aGVA) Release 15/05/2020,Approximate gross value added at basic prices (aGVA) Release 24/06/2021,Approximate gross value added at basic prices (aGVA) Revision,Approximate gross value added at basic prices (aGVA) % Revision,"Total purchases of goods, materials and services Release 15/05/2020","Total purchases of goods, materials and services Release 24/06/2021","Total purchases of goods, materials and services Revision","Total purchases of goods, materials and services % Revision"
0,A-S (Part) 1,"Agriculture, Fishing, Production,",North East,100405,101551,1146,1.1,32914,33358,443,1.3,67385,67926,541,0.8
1,A-S (Part) 1,"Construction, Distribution and",North West,349232,364183,14952,4.3,121577,123107,1530,1.3,225094,237545,12451,5.5
2,A-S (Part) 1,Non-Financial Service Industries,Yorkshire and The Humber,238345,253567,15223,6.4,76847,80367,3520,4.6,160594,172425,11831,7.4
3,A-S (Part) 1,Non-Financial Service Industries,East Midlands,207770,222562,14792,7.1,71010,73379,2369,3.3,137820,150088,12268,8.9
4,A-S (Part) 1,Non-Financial Service Industries,West Midlands,293216,300906,7690,2.6,97737,94488,-3249,-3.3,194166,205137,10972,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,H-S 1,Non-Financial Service industries,Wales,28571,31166,2595,9.1,14177,16618,2441,17.2,14595,14882,287,2.0
86,H-S 1,Non-Financial Service industries,Scotland,77234,82193,4959,6.4,41636,43177,1540,3.7,36586,39933,3347,9.1
87,H-S 1,Non-Financial Service industries,Great Britain,1377144,1433797,56653,4.1,686333,709470,23137,3.4,695391,728924,33533,4.8
88,H-S 1,Non-Financial Service industries,Northern Ireland,16103,17108,1006,6.2,8390,9686,1295,15.4,7918,7737,-181,-2.3


In [11]:
# table = next(df_iterator)
subdf, rows, cols, relevant_rows, relevant_columns, pairwise_matrices, contents = find_relevant_content(_cloze, table, vectorizer, nlp, stopwords, return_empty = False) 

In [12]:
print(subdf.shape, table.shape)

(90, 1) (90, 15)


In [13]:
table

,A-S (Part) 1,"Agriculture, Fishing, Production,",Country and Region North East,Total turnover Release 15/05/2020,Total turnover Release 24/06/2021,Total turnover Revision,Total turnover % Revision,Approximate gross value added at basic prices (aGVA) Release 15/05/2020,Approximate gross value added at basic prices (aGVA) Release 24/06/2021,Approximate gross value added at basic prices (aGVA) Revision,Approximate gross value added at basic prices (aGVA) % Revision,"Total purchases of goods, materials and services Release 15/05/2020","Total purchases of goods, materials and services Release 24/06/2021","Total purchases of goods, materials and services Revision","Total purchases of goods, materials and services % Revision"
0,A-S (Part) 1,"Agriculture, Fishing, Production,",North East,100405,101551,1146,1.1,32914,33358,443,1.3,67385,67926,541,0.8
1,A-S (Part) 1,"Construction, Distribution and",North West,349232,364183,14952,4.3,121577,123107,1530,1.3,225094,237545,12451,5.5
2,A-S (Part) 1,Non-Financial Service Industries,Yorkshire and The Humber,238345,253567,15223,6.4,76847,80367,3520,4.6,160594,172425,11831,7.4
3,A-S (Part) 1,Non-Financial Service Industries,East Midlands,207770,222562,14792,7.1,71010,73379,2369,3.3,137820,150088,12268,8.9
4,A-S (Part) 1,Non-Financial Service Industries,West Midlands,293216,300906,7690,2.6,97737,94488,-3249,-3.3,194166,205137,10972,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,H-S 1,Non-Financial Service industries,Wales,28571,31166,2595,9.1,14177,16618,2441,17.2,14595,14882,287,2.0
86,H-S 1,Non-Financial Service industries,Scotland,77234,82193,4959,6.4,41636,43177,1540,3.7,36586,39933,3347,9.1
87,H-S 1,Non-Financial Service industries,Great Britain,1377144,1433797,56653,4.1,686333,709470,23137,3.4,695391,728924,33533,4.8
88,H-S 1,Non-Financial Service industries,Northern Ireland,16103,17108,1006,6.2,8390,9686,1295,15.4,7918,7737,-181,-2.3


In [14]:
_relevant_dfs[1].replace('/', '_')[1:]

'businessindustryandtrade_business_businessservices_datasets_uknonfinancialbusinesseconomyannualbusinesssurveyrevisionsandchangeonpreviousyear'

In [15]:
table = next(df_iterator)
table

IMPUTE_COL_NANS: True
DROP_UNNAMED: True
BACKFILL_HEADERS: True
FILL_NA: True
IMPUTE_FIRST_COL_EMPTY: True
IMPUTE_ALL_COL_EMPTY: True


,Standard Industrial Classification (Revised 2007) Section,"Agriculture, Fishing, Production,",Country and Region,Year,Total turnover,Change on previous year,% change on previous year,Approximate gross value added at basic prices (aGVA),Change on previous year,% change on previous year,"Total purchases of goods, materials and services",Change on previous year,% change on previous year
0,A-S (Part) 1,"Agriculture, Fishing, Production,",North East,2008,84408,NaN,NaN,28037,NaN,NaN,55883,NaN,NaN
1,A-S (Part) 1,"Construction, Distribution and",North East,2009,79257,-5150.0,-6.1,25178,-2859.0,-10.2,53179,-2704.0,-4.8
2,A-S (Part) 1,Non-Financial Service Industries,North East,2010,84482,5225.0,6.6,26081,902.0,3.6,57799,4620.0,8.7
3,A-S (Part) 1,Non-Financial Service Industries,North East,2011,86327,1845.0,2.2,26812,732.0,2.8,59668,1869.0,3.2
4,A-S (Part) 1,Non-Financial Service Industries,North East,2012,86368,41.0,2.2,28755,1942.0,7.2,56835,-2832.0,-4.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,H-S (Part)1,Non-Financial Service industries,North East,2015,1250922,62314.0,5.2,661711,43289.0,7.0,594685,18205.0,3.2
992,H-S (Part)1,Non-Financial Service industries,North East,2016,1321795,70874.0,5.7,680344,18633.0,2.8,646459,51774.0,8.7
993,H-S (Part)1,Non-Financial Service industries,North East,2017,1393247,71451.0,5.4,694723,14379.0,2.1,703309,56850.0,8.8
994,H-S (Part)1,Non-Financial Service industries,North East,2018,1450905,57658.0,4.1,719156,24432.0,3.5,736661,33352.0,4.7


In [16]:
table.to_csv('test.csv')

In [17]:
unigram_vectorizer = TfidfVectorizer(ngram_range= (1,1), stop_words='english')
bigram_vectorizer = TfidfVectorizer(ngram_range= (2,2), stop_words='english')
trigram_vectorizer = TfidfVectorizer(ngram_range= (3,3), stop_words='english')
bitrigram_vectorizer = TfidfVectorizer(ngram_range= (2,3), stop_words='english')

In [18]:
example = [_cloze] + table.columns.tolist()
for vectorizer in [unigram_vectorizer, bigram_vectorizer, trigram_vectorizer, bitrigram_vectorizer]:
    pairwise = vectorizer.fit_transform(example)
    # pairwise = vectorizer.fit_transform(example)
    out = (pairwise * pairwise.T).toarray()[0]
    [print(example[i], ':', x) for i, x in enumerate(out) if x != 0.0]
    print(vectorizer.ngram_range, out, '\n')

West Midlands, Yorkshire and The Humber, Scotland and East Midlands, were the four regions decreasing year-on-year in aGVA; the largest percentage decrease was in West Midlands where aGVA fell by £2.5 billion (2.6%), from £94.5 billion to £92 billion. : 1.0000000000000004
Year : 0.1625825238894538
Change on previous year : 0.08428707551667357
% change on previous year : 0.08428707551667357
Approximate gross value added at basic prices (aGVA) : 0.09355279423885791
Change on previous year : 0.08428707551667357
% change on previous year : 0.08428707551667357
Change on previous year : 0.08428707551667357
% change on previous year : 0.08428707551667357
(1, 1) [1.         0.         0.         0.         0.16258252 0.
 0.08428708 0.08428708 0.09355279 0.08428708 0.08428708 0.
 0.08428708 0.08428708] 

West Midlands, Yorkshire and The Humber, Scotland and East Midlands, were the four regions decreasing year-on-year in aGVA; the largest percentage decrease was in West Midlands where aGVA fell 

In [16]:
example

NameError: name 'example' is not defined

In [20]:
example = ['my cat is so amazing', 'my dog is an asshole', 'the purple lamborghini', 'this house is huge, it has a cat','cat are amazing', 'cats are asshole']

In [21]:
X = unigram_vectorizer.fit_transform(example)

In [22]:
print((X * X.T).toarray())

[[1.         0.         0.         0.28363941 1.         0.        ]
 [0.         1.         0.         0.         0.         0.40206531]
 [0.         0.         1.         0.         0.         0.        ]
 [0.28363941 0.         0.         1.         0.28363941 0.        ]
 [1.         0.         0.         0.28363941 1.         0.        ]
 [0.         0.40206531 0.         0.         0.         1.        ]]


In [23]:
nlp = spacy.load('en_core_web_trf')

In [24]:
example = [
    'West Midlands, Yorkshire and The Humber, Scotland and East Midlands, were the four regions decreasing year-on-year in approximate gross value added aGVA; the largest percentage decrease was in West Midlands where aGVA fell by £2.5 billion (2.6%), from £94.5 billion to £92 billion.', 
    'Agriculture, Fishing, Production,', 
    'Agriculture, Fishing, Production,', 
    'Construction, Distribution and',
    'Non-Financial Service Industries', 
    'Agriculture, forestry and fishing', 
    'Production industries', 
    'Construction industries', 
    'Distribution industries', 
    'Non-Financial Service industries']

sav = [nlp(x) for x in example]

In [25]:
len([x.lemma_ for x in sav[0]])

59

In [26]:
len(example[0].split(' '))

43

In [27]:
np.unique([x.lemma_ for x in sav[0]])

array(['%', '(', ')', ',', '-', '.', '2.5', '2.6', '92', '94.5', ';',
       'East', 'Humber', 'Midlands', 'Scotland', 'West', 'Yorkshire',
       'add', 'agva', 'and', 'approximate', 'be', 'billion', 'by',
       'decrease', 'decreasing', 'fall', 'four', 'from', 'gross', 'in',
       'large', 'on', 'percentage', 'region', 'the', 'to', 'value',
       'where', 'year', '£'], dtype='<U11')

In [12]:
clozes_df.iloc[0].source_text

'In 2019, approximate gross value added at basic prices (aGVA) of the UK non-financial business economy was estimated to be £1,313.9 billion; an increase of £42.8 billion (3.4%) compared with 2018.'

In [19]:
_cloze

'West Midlands, Yorkshire and The Humber, Scotland and East Midlands, were the four regions decreasing year-on-year in aGVA; the largest percentage decrease was in West Midlands where aGVA fell by £2.5 billion (2.6%), from £94.5 billion to £92 billion.'

In [20]:
clozes_df.iloc[20]

ids                     034d226c730270fae716bdb43bc8a571daf63dc9
cloze_text     West Midlands, Yorkshire and The Humber, Scotl...
source_text    West Midlands, Yorkshire and The Humber, Scotl...
answer_text                                        East Midlands
answer_type                                                  LOC
data           [/businessindustryandtrade/business/businessse...
Name: 20, dtype: object

In [24]:
df.iloc[4].bulletin

'businessindustryandtrade/business/businessservices/bulletins/nonfinancialbusinesseconomyukandregionalannualbusinesssurvey/2019finalresults'

In [27]:
clozes_df.iloc[20].data[0].replace('/', '_')[1:]



'businessindustryandtrade_business_businessservices_datasets_uknonfinancialbusinesseconomyannualbusinesssurveyregionalresultsqualitymeasures'

In [ ]:
businessindustryandtrade_business_businessservices_datasets_uknonfinancialbusinesseconomyannualbusinesssurveyregionalresultsqualitymeasures.xls